In [1]:
import csv
import os

import networkx as nx
import nltk
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer

In [2]:
ingredients = []
with open('data/ingredients/all_ingredients.txt') as f:
    for line in f:
        ingredient = line.strip()
        ingredients.append(ingredient)
ingredients = sorted(ingredients, key=len, reverse=True)

In [3]:
ingredients[:10]

['high altitude adjustment for deep fat frying',
 "mrs dash's fiesta lime salt-free seasoning",
 'high altitude adjustment for cookie',
 'trappist or monastery-style cheese',
 'high altitude baking quick bread',
 "i can't believe it's not butter",
 'meat: safe cooking temperature',
 'african birdseye chile pepper',
 'heirloom weight & measurement',
 'trompette de la mort mushroom']

In [4]:
techniques = []
with open('data/techniques/all_techniques.txt') as f:
    for line in f:
        technique = line.strip()
        techniques.append(technique)
techniques = sorted(techniques, key=len, reverse=True)

In [5]:
techniques[:10]

['thermal immersion circulator',
 'extreme-heat stir-fry',
 'flash pasteurization',
 'amylolytic process',
 'biomass briquettes',
 'coagulated protein',
 'high heat stir fry',
 'mongolian barbecue',
 'culinary triangle',
 'food preservation']

In [6]:
def singularize(word):
    if word == 'cookies':
        result = 'cookie'
    elif word == 'mrs':
        result = 'mrs'
    elif word == 'ras':
        result = 'ras'
    elif word == 'somen':
        result = 'somen'
    elif word == 'cos':
        result = 'cos'
    elif word == 'monks':
        result = "monk's"
    elif word == 'webbs':
        result = "webb's"
    else:
        result = WordNetLemmatizer().lemmatize(word.lower())
    return result

In [7]:
def inf_ing(v_ing):
    inf = v_ing
    if v_ing == 'de-seeding':
        inf = 'de-seed'
    elif v_ing == 'de-boning':
        inf = 'de-bone'
    elif v_ing == 'de-bearding':
        inf = 'de-beard'
    elif v_ing == 'gutting':
        inf = 'gut'
    elif v_ing == 'de-glazing':
        inf = 'de-glaze'
    elif v_ing == 'sautéing':
        inf = 'sauté'
    elif v_ing == 'degorging':
        inf = 'degorge'
    elif v_ing == 'stir-frying':
        inf = 'stir-fry'
    elif v_ing == 'broasting':
        inf = 'broast'
    elif v_ing == 'juicing':
        inf = 'juice'
    elif v_ing == 'parbaking':
        inf = 'parbake'
    elif v_ing == 'charbroiling':
        inf = 'charbroil'
    elif v_ing == 'crinkle-cutting':
        inf = 'crinkle-cut'
    elif v_ing == 'cheesemaking':
        inf = 'cheesemake'
    elif v_ing == 'swissing':
        inf = 'swiss'
    elif v_ing == 'flash-frying':
        inf = 'flash-fry'
    elif v_ing == 'flashbaking':
        inf = 'flashbake'
    else:
        inf = WordNetLemmatizer().lemmatize(v_ing.lower(), 'v')
    return inf

def infinitive(words):
    aux = []
    word_list = words.split()
    for word in word_list:
        if word != 'cook' and word != 'cooking':
            if word.endswith('ing'):
                inf = inf_ing(word.lower())
                aux.append(inf)
#                 if inf == word:
#                     print('NO INFINITIVE FOR', word)
            else:
                aux.append(word.lower())
    result = ' '.join(aux)
    return result

In [8]:
# load ingr_id_graph

In [16]:
soup.select('div.title-source h1[itemprop=name]')[0]

<h1 itemprop="name">
        Corn Pudding
       </h1>

In [27]:
with open('data/ingredients/epicurious_ingr_ids.csv', 'w') as f1,\
     open('data/techniques/epicurious_techniques.csv', 'w') as f2,\
     open('data/epicurious_ingredient_technique_log.txt', 'w') as log:
    writer1 = csv.writer(
        f1,
        delimiter=',',
        quotechar='"',
        quoting=csv.QUOTE_MINIMAL
    )
    writer2 = csv.writer(
        f2,
        delimiter=',',
        quotechar='"',
        quoting=csv.QUOTE_MINIMAL
    )
    writer3 = csv.writer(
        log,
        delimiter=',',
        quotechar='"',
        quoting=csv.QUOTE_MINIMAL
    )
#     path = '/media/antonio/WD1T/datasets allrecipes and epicurious/epicurious/'
#     path = 'data/epicurious/'
    path = 'data/allrecipes/'
    count = 0
    for folder_number in sorted(map(int, os.listdir(path))):
        folder_name = str(folder_number)
        folder_path = path + folder_name
        for file_number in sorted(map(lambda x: int(x[:-5]), os.listdir(folder_path))):
            file_name = str(file_number) + '.html'
            file_path = folder_path + '/' + file_name
            with open(file_path) as f:
                soup = BeautifulSoup(f.read(), 'html.parser')
#                 recipe_info = soup.find('recipe-signup')
#                 recipe_id = str(file_number)
#                 recipe_title = soup.select('div.title-source h1[itemprop=name]')[0].string.strip()
#                 info_row = [recipe_id, recipe_title]
                
                ingr_set = set()
#                 ingredient_amounts = soup.select('div.ingredients-info li[itemprop=ingredients]')
                ingredient_amounts = soup.select('section.recipe-ingredients span[itemprop=ingredients]')
                for ingredient_amount in ingredient_amounts:
                    content = ingredient_amount.text.strip()
                    if content:
                        tokens = nltk.word_tokenize(content)
                        singularized = ' '.join(map(singularize, tokens))
                        found_ingr = False
                        for ingredient in ingredients:
                            if ingredient in singularized:
                                print(singularized, '=>', ingredient)
                                found_ingr = True
                                ingr_set.add(ingredient)
                        if not found_ingr:
                            writer3.writerow(['INGREDIENT', file_path, singularized])
#                 ingredient_row = info_row + list(ingr_set)
#                 writer1.writerow(ingredient_row)
#                 print(ingredient_row)
                print()
                
#                 technique_names = set()
#                 instructions = soup.select('section.recipe-directions span.recipe-directions__list--item')
#                 for instruction in instructions:
#                     content = instruction.text.strip()
#                     if content:
#                         tokens = nltk.word_tokenize(content)
#                         infinitived = ' '.join(map(infinitive, tokens))
#                         found_tech = False
#                         for technique in techniques:
#                             if technique in infinitived:
#                                 found_tech = True
#                                 technique_names.add(technique)
#                         if not found_tech:
#                             writer3.writerow(['TECHNIQUE', file_path, infinitived])
#                 technique_row = info_row + list(technique_names)
#                 writer2.writerow(technique_row)
                
#                 count += 1
#                 if count % 10000 == 0:
#                     print(count, 'recipes processed.')

2 egg => egg
1/2 cup vegetable oil => vegetable oil
1/2 cup vegetable oil => vegetable
1/2 cup vegetable oil => oil
1 cup white sugar => sugar
1 cup grated zucchini => zucchini
1/2 cup crushed pineapple , drained => pineapple
1/2 cup crushed pineapple , drained => apple
1 teaspoon vanilla extract => vanilla extract
1 teaspoon vanilla extract => vanilla
1 teaspoon vanilla extract => tea
2 cup all-purpose flour => flour
1 teaspoon baking soda => baking soda
1 teaspoon baking soda => soda
1 teaspoon baking soda => tea
1/2 teaspoon baking powder => baking powder
1/2 teaspoon baking powder => tea
1/2 teaspoon salt => salt
1/2 teaspoon salt => tea
3/4 teaspoon ground cinnamon => cinnamon
3/4 teaspoon ground cinnamon => tea
1/4 teaspoon ground nutmeg => nutmeg
1/4 teaspoon ground nutmeg => nut
1/4 teaspoon ground nutmeg => tea
1/2 cup chopped walnut => walnut
1/2 cup chopped walnut => chop
1/2 cup chopped walnut => hop
1/2 cup chopped walnut => nut

1 1/2 teaspoon active dry yeast => yeast
1 

In [ ]:
convertir ambos en listas de palabras y comprobar q una lista esta contenida en la otra. si hay varias opciones de varios ingredientes, desechar listas pequenas contenidas en otras mayores.

In [ ]:
ingredient_amount.text.strip()

In [ ]:
nx.write_gexf(ingr_id_graph, 'data/ingredients/ingr_id_graph.gexf')

In [ ]:
# with open('data/allrecipes/6000/6903.html') as f:
#     soup = BeautifulSoup(f.read(), 'html.parser')
#     info = soup.find('recipe-signup')
#     print(info['data-id'])
#     print(info['data-title'])
#     qq=info['data-title']
#     info_row = [info['data-id'], info['data-title']]
#     technique_names = []
#     instructions = soup.select('section.recipe-directions span.recipe-directions__list--item')
#     for instruction in instructions:
#         infinitived = ' '.join(map(infinitive, instruction.string.strip().split()))
#         for technique in techniques:
#             if technique in infinitived:
#                 technique_names.append(technique)
#     technique_row = info_row + technique_names
#     print(technique_row)
#     print()

In [ ]:
# las tecnicas no tienen pq ser solo verbos - pueden ser sustantivos, por ejemplo, bbq, fondue...

In [ ]:
# for folder_name in os.listdir('data/allrecipes/'):
#     folder_path = 'data/allrecipes/' + folder_name
#     for file_name in os.listdir(folder_path):
#         file_path = folder_path + '/' + file_name
#         with open(file_path) as f:
#             soup = BeautifulSoup(f.read(), 'html.parser')
#             info = soup.find('recipe-signup')
#             print(info['data-id'])
#             print(info['data-title'])
#             ingredients = soup.select('span[itemprop=ingredients]')
#             for ingredient in ingredients:
#                 print(ingredient.string.strip())
#             print()